In [1]:
import pandas as pd
import csv
import nltk
import xml.etree.ElementTree as ET
from nltk.util import everygrams

In [2]:
df = pd.read_csv('the csv file of 500 unique FB ads')

In [3]:
df.head()

,candidate_name,date,post_id,text,Attack (Ania),spanish marked
0,Martin O'Malley,2011-08-04 16:30:15,28684115392_180756128656528,"Our nation needs more jobs, not less. When tea...",1,NaN
1,Ajamu Baraka,2016-11-01 16:30:00,544311182270733_1160178944017284,"An op-ed written by Jill Stein and I. ""What ha...",1,NaN
2,Scott Walker,2016-06-03 22:47:46,138492335404_10156950994675405,FACT CHECK. As Wisconsin Democrats hold their ...,0,NaN
3,Rand Paul,2015-08-01 0:30:00,54172246106_10153244002971107,Two days.\n \nThat's how long you and I have t...,0,NaN
4,Donald J. Trump,2016-03-04 2:33:25,153080620724_10156714837560725,The last person that Hillary Clinton wants to ...,0,NaN


In [4]:
# get the classifier lexicon
xml_doc = 'lexicon.txt'
tree = ET.parse(xml_doc)
root = tree.getroot() # * this is the extractor for xml doc  .getroot()
topic_dict = {}

for category in root:
    f = category.get('name')
    keylist=[]
    for key in category:
        key_name = key.get('name')
        keylist.append(key_name)
    
    topic_dict[f]=keylist

topics = sorted(topic_dict)



In [6]:
topics

['Economic',
 'Environment',
 'Foreign policy',
 'Immigration',
 'Military',
 'Safety',
 'Social Programs',
 'Social and Cultural']

In [7]:
#print(topic_dict)

In [11]:
for t in topics: 
    string1 = '\n'.join(topic_dict[t])
    string2 = t +': '+ '\n\n' + string1+'\n\n\n'
    print(string2)

Economic: 

taxpayers
taxes
trade
taxpayers
Budget Deal
Fair Tax
Fight For 15
growth code
made in america
MFG Day
Raise The Wage
tax reform
buy American
capital gains
CEOs
corp profits
credit rating
debt ceiling
economic imperative
enterprise
entrepreneurial
federal labor law
Federal Reserve
flat tax plan
Gilmore Growth Code
growth agenda
income inequality
infrastructure
jobs
labor law
loan
low-wage
overtime
overtime pay
overtime work
personal property taxes
pro worker
pro-worker
productivity
prosperity
real estate
roads
tax code
tax plan
tax rate
tax system
top 1%
transportation
vocational training
Wall Street CEOs
workplace protections
zero-based budgeting
aaa bond
accountants
adjusting monetary
after-tax
alan greenspan
alan krueger
already overburdened
american companies
american corporations
american dream
american household
american jobs
american products
american worker
american workers
anti-monopoly
antitrust
assets
auto industry
auto supplier
automakers
automobile industry
auto

In [24]:
lexicon = pd.DataFrame.from_dict(topic_dict,orient='index')
lexicon = lexicon.transpose()
#lexicon.head()
lexicon.to_excel('the_new_lexicon.xlsx',index=False)

In [7]:
# find the words in each message
def find_topic(x):
    token = [w for w in nltk.word_tokenize(x)]
    words = [w for w in token if w.isalpha()]
    # tokenize the ngrams from 1 to 7 words
    ngrams = [' '.join(w) for w in everygrams(words,min_len=1,max_len=7)]
    labels = []
    
    for t in topics:
              
        if set(ngrams)&set(topic_dict[t]):
            labels.append(t)
            
        else:
            labels.append('nan') 
    
    labels = [x for x in labels if str(x) != 'nan']
    
    if labels:
        return ','.join(labels)
    else:
        return "no topic"
        

In [8]:
# testing texts 
s1= 'with open "attack_ad_topic.csv", w ,encoding= Latin1 as output_csv'
s2 = 'marriage licenses sex marriage klu klux partial birth abortion, religious freedom same sex couples'

In [10]:
print(find_topic(s2))

Social and Cultural


In [11]:
df['topic label'] = df['text'].apply(lambda x: find_topic(x))

In [12]:
df.head()

,candidate_name,date,post_id,text,Attack (Ania),spanish marked,topic label
0,Martin O'Malley,2011-08-04 16:30:15,28684115392_180756128656528,"Our nation needs more jobs, not less. When tea...",1,NaN,"Economic,Foreign policy"
1,Ajamu Baraka,2016-11-01 16:30:00,544311182270733_1160178944017284,"An op-ed written by Jill Stein and I. ""What ha...",1,NaN,Social Programs
2,Scott Walker,2016-06-03 22:47:46,138492335404_10156950994675405,FACT CHECK. As Wisconsin Democrats hold their ...,0,NaN,"Economic,Social Programs"
3,Rand Paul,2015-08-01 0:30:00,54172246106_10153244002971107,Two days.\n \nThat's how long you and I have t...,0,NaN,"Economic,Social and Cultural"
4,Donald J. Trump,2016-03-04 2:33:25,153080620724_10156714837560725,The last person that Hillary Clinton wants to ...,0,NaN,no topic


In [13]:
# save the file 
df.to_csv('attack_ad_topic2.csv', index= False)

In [15]:
print(len(df))
len(df[df['topic label']=='no topic']) 
# In totall there are 300 samples.
# 118 of them do not contain the topics we listed and 182 of them has at least one topic 

300


118